# SFv2 Template notebook

## Import modules

In [ ]:
import os
import sys

data_path=os.path.abspath('C:/Users/hcarv/Documents/work/Protein Design 2021/Group1') #Where your group data is
base_path=os.path.abspath('C:/Users/hcarv/Documents/GitHub/SimFound_v2/source/') #Where your source code is (SFv2)

sys.path.append(base_path)
sys.path.append(data_path)

import importlib

import SETD2_protocols as P


# Protocols

In [57]:
pdb='5a71a.pdb'

importlib.reload(P)
protocol=P.Protocols(workdir=data_path)

system=protocol.pdb2omm(input_pdbs=pdb, 
                          extra_ff_files=['gaff-2.11modv3_1.xml'], 
                          protonate=True,
                          solvate=True,
                          inspect=True) 

IndentationError: unexpected indent (SETD2_protocols.py, line 372)

In [56]:
import nglview
print(protocol.out_pdb)
view=nglview.show_structure_file('C:/Users/hcarv/Documents/work/Protein Design 2021/Group1/test.pdb')
view

C:\Users\hcarv\Documents\work\Protein Design 2021\Group1/test.pdb


NGLWidget()

In [44]:
view

NGLWidget()

In [26]:
simulation=protocol.setSimulation()